## CAI - Colony Avalanche Index - Rebalancing data for April 2024

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
date = '2024-04-01'
min_mcap =  0.1e8
min_weight = 0.005
max_weight = 0.5
circ_supply_threshold = 0.3
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["avalanche-ecosystem"]

# Calculating max slippage based on liquidity threshold
liq = 2*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                date,
                                1,
                                "avalanche",
                                "0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0",
                                "cai_benchmark_data",
                                "cai_liquidities",
                                min_mcap,
                                min_weight,
                                max_weight,
                                100000,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                -0.12,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe, Platypus, Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    single_chain="avalanche",
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[["avalanche-2"],["wrapped-avax"]],
    add_category_assets=["shrapnel-2"],
    remove_category_assets=['wrapped-bitcoin',"gmx","pangolin"],
    values_to_update=[{'id':'pangolin','category':'total_supply','value':230000000},
                      {'id':'benqi-liquid-staked-avax','category':'market_cap','value':20000000000},
                      {'id':'benqi-liquid-staked-avax','category':'symbol','value':"sAVAX"}
                      ],
    weight_split_data={"asset_to_split":"wrapped-avax","asset_to_receive":"benqi-liquid-staked-avax","split_ratio":0.6}
    )
liquidity

Excluding coq-inu, pricing data available only for 147 < 180 days
Excluding ox-fun, pricing data available only for 73 < 180 days
Excluding miracle-play, pricing data available only for 178 < 180 days
Excluding raini-studios-token, pricing data available only for 158 < 180 days
Excluding platypus-usd, pricing data available only for 13 < 180 days
Excluding kimbo, pricing data available only for 136 < 180 days
Excluding avax-has-no-chill, pricing data available only for 129 < 180 days
Excluding gogopool-ggavax, pricing data available only for 141 < 180 days
Excluding any-inu, pricing data available only for 122 < 180 days
Table created successfully
Table created successfully
Manually calculating slippage for uniswap on avalanche
Manually calculating slippage for maker on avalanche
havven on avalanche returned an invalid API response
havven recorded no valid response across any platform
axelar on avalanche returned an invalid API response
axelar recorded no valid response across any plat

,avalanche,optimal chain,best slippage,best slippage chain
id,,,,
wrapped-avax,-0.002948,avalanche,-0.002948,avalanche
benqi-liquid-staked-avax,-0.003828,avalanche,-0.003828,avalanche
bitcoin-avalanche-bridged-btc-b,-0.006568,avalanche,-0.006568,avalanche
euro-coin,-0.023004,avalanche,-0.023004,avalanche
joe,-0.042355,avalanche,-0.042355,avalanche
avalaunch,-0.061496,avalanche,-0.061496,avalanche
benqi,-0.098335,avalanche,-0.098335,avalanche
chainlink,-0.236032,None,-0.236032,avalanche
colony,-0.262452,None,-0.262452,avalanche


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
benqi-liquid-staked-avax,sAVAX,2.000000e+10,0.300000,77,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,avalanche
wrapped-avax,AVAX,1.277980e+10,0.200000,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,avalanche
joe,JOE,1.692316e+08,0.313398,80,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,avalanche
benqi,QI,6.783487e+07,0.125623,32,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,avalanche
avalaunch,XAVA,3.292846e+07,0.060980,15,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4,avalanche


## Contract Output

In [5]:
methodology_class.output_for_contract()

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4
77,80,32,51,15
